# Proyek Analisis Data: Bike Sharing Dataset
- **Nama:** Silvia Dharma
- **Email:** silviadharma07@gmail.com
- **ID Dicoding:** silvia_dharma_B3Fq

## Menentukan Pertanyaan Bisnis

- Bagaimana tren jumlah penyewaan sepeda dalam beberapa tahun terakhir?
- Apakah kondisi cuaca mempengaruhi jumlah pengguna sepeda?
- Bagaimana perbedaan pola penggunaan sepeda antara hari kerja (workingday), hari libur (holiday), dan hari biasa (weekday)?
- Apakah ada korelasi antara suhu yang menunjukkan kondisi ketika jumlah penyewaan sepeda tinggi?
- Apakah musim mempengaruhi jumlah penyewa sepeda berdasarkan kategori Casual dan Registered

## Import Semua Packages/Library yang Digunakan

In [ ]:
# Libraries Used

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from datetime import datetime

## Data Wrangling

### Gathering Data

In [ ]:
# Load "day" table

day_df = pd.read_csv("https://raw.githubusercontent.com/slvyarc/Bike-Sharing_Dicoding/main/Dataset/day.csv")
day_df.head()

In [ ]:
# Load "hour" table

hour_df = pd.read_csv("https://raw.githubusercontent.com/slvyarc/Bike-Sharing_Dicoding/main/Dataset/hour.csv")
hour_df.head()

### Assessing Data

**Menilai day_df**

In [ ]:
# Check the data types of day_df
day_df.info()

Ada 1 tipe data yang tidak sesuai (dteday-object). Data tersebut perlu diperbaiki menjadi dteday -> datetime

In [ ]:
# Check for missing values in day_df
day_df.isna().sum()

Dapat disimpulkan bahwa tidak terdapat missing value pada tabel "day_df"

In [ ]:
#Check for duplicated data in day_df.
print("Jumlah duplikasi: ", day_df.duplicated().sum())

Duplikasi 0 artinya tidak ada duplikasi data

In [ ]:
#Displaying Summary Statistical Parameters.

day_df.describe()

Tidak ada masalah pada parameter statistik (aman)

**Menilai "hour_df"**

In [ ]:
# Check the data types of hour_df
hour_df.info()

Ada 1 tipe data yang tidak sesuai (dteday-object). Data tersebut perlu diperbaiki menjadi dteday -> datetime

In [ ]:
# Check for missing values in hour_df
hour_df.isna().sum()

Tidak ada duplikasi data (aman)

In [ ]:
#Displaying Summary Statistical Parameters.

hour_df.describe()

Tidak ada masalah pada parameter statistik (aman)

### Cleaning Data

**Menghapus tabel yang tidak diperlukan**

In [ ]:
# Deleting the "hour_df" table as it is not relevant to the business question and its contents are the same as the "day_df" table, differing only in the "hr" column.
del hour_df


**Menghapus column yang tidak diperlukan**

In [ ]:
# Removing the instant column (does not provide meaningful information)
# Removing the windspeed column (not relevant to the business question)
drop_columns = ['instant', 'windspeed']

for col in day_df.columns:
    if col in drop_columns:
        day_df.drop(labels=col, axis=1, inplace=True)

day_df.head()


**Modifikasi column**

In [ ]:
# Changing column names (Optional)
day_df.rename(columns={
    'dteday': 'dateday',
    'yr': 'year',
    'mnth': 'month',
    'cnt': 'count'
}, inplace=True)

day_df.head()


In [ ]:
# Changing the data type of the dateday column to datetime.
day_df['dateday'] = pd.to_datetime(day_df['dateday'])
day_df.info()


In [ ]:
day_df.head()

In [ ]:
# Changing data types
# Weekday, month, year columns
day_df['weekday'] = day_df['dateday'].dt.day_name()
day_df['year'] = day_df['dateday'].dt.year

# Season column
day_df['season'] = day_df['season'].map({
    1: 'Spring', 2: 'Summer', 3: 'Fall', 4: 'Winter'
})

# Weathersit column
day_df['weathersit'] = day_df['weathersit'].map({
    1: 'Clear/Partly Cloudy',
    2: 'Misty/Cloudy',
    3: 'Light Snow/Rain',
    4: 'Severe Weather'
})

day_df.head()


In [ ]:
day_df.dtypes

In [ ]:
# Resampling data based on month and calculating total rides
monthly_rent_df = day_df.resample(rule='M', on='dateday').agg({
    "casual": "sum",
    "registered": "sum",
    "count": "sum"
})

# Change index format to month-year (Jan-20, Feb-20, etc.)
monthly_rent_df.index = monthly_rent_df.index.strftime('%b-%y')
monthly_rent_df = monthly_rent_df.reset_index()

# Rename columns
monthly_rent_df.rename(columns={
    "dateday": "yearmonth",
    "count": "total_rides",
    "casual": "casual_rides",
    "registered": "registered_rides"
}, inplace=True)

monthly_rent_df


**Save data yang telah dicleaning**

In [ ]:
day_df.to_csv("cleaned_bikeshare.csv", index=False)

## Exploratory Data Analysis (EDA)

### Explore data day_df

**Analisis Statistik Penyewaan Sepeda Berdasarkan Bulan (Month)**

In [ ]:
# Grouping bike renters (casual and registered) data by month
grouped_by_month = day_df.groupby('month')

# Calculating aggregate statistics for each month
aggregated_stats_by_month = grouped_by_month['count'].agg(['max', 'min', 'mean', 'sum'])
aggregated_stats_by_month


 Dapat disimpulkan bahwa Bulan Juni sebagai bulan dengan jumlah penyewaan sepeda tertinggi, baik dari segi rata-rata maupun total keseluruhan. Sebaliknya, bulan Januari merupakan bulan dengan aktivitas penyewaan sepeda yang paling minim

**Analisis Statistik Penyewaan Sepeda Berdasarkan Cuaca (Weathersit)**

In [ ]:
# Grouping bike renters (casual and registered) data by weather
grouped_by_weather = day_df.groupby('weathersit')

# Calculating aggregate statistics for each weather type
aggregated_stats_by_weather = grouped_by_weather['count'].agg(['max', 'min', 'mean', 'sum'])
aggregated_stats_by_weather


Kondisi cuaca yang cerah atau sebagian cerah ("Clear/Partly Cloudy") cenderung memiliki jumlah penyewa sepeda yang lebih tinggi, diikuti oleh kondisi cuaca "Misty/Cloudy", sementara kondisi cuaca "Light Snow/Rain" memiliki jumlah penyewa sepeda yang paling rendah.

**Analisis Statistik Penyewaan Sepeda Berdasarkan Hari Libur (Holiday)**

In [ ]:
# Grouping bike renters (casual and registered) data by holiday
grouped_by_holiday = day_df.groupby('holiday')

# Calculating aggregate statistics for each holiday condition
aggregated_stats_by_holiday = grouped_by_holiday['count'].agg(['max', 'min', 'mean', 'sum'])
aggregated_stats_by_holiday


 Saat bukan hari libur (non-holiday), jumlah penyewa sepeda cenderung lebih tinggi dibandingkan saat hari libur (holiday).

**Perbandingan Aktivitas Penyewaan Sepeda: Hari Kerja vs. Akhir Pekan**

In [ ]:
# Comparing the number of bike renters on weekdays and weekends
grouped_by_weekday = day_df.groupby('weekday')

# Calculating aggregate statistics for the number of bike renters on weekdays and weekends
aggregated_stats_by_weekday = grouped_by_weekday['count'].agg(['max', 'min', 'mean'])
aggregated_stats_by_weekday


Dari data yang terlihat, urutan rata-rata jumlah penyewa sepeda dari yang paling banyak ke yang paling sedikit adalah: Jumat (Fri), Kamis (Thu), Sabtu (Sat), Rabu (Wed), Selasa (Tue), Senin (Mon), dan Minggu (Sun).

**Perbandingan Jumlah Penyewa Sepeda antara Hari Kerja dan Bukan Workingday**

In [ ]:
# Grouping bike renters data by working day
grouped_by_workingday = day_df.groupby('workingday')

# Calculating aggregate statistics for the number of bike rentals on working and non-working days
aggregated_stats_by_workingday = grouped_by_workingday['count'].agg(['max', 'min', 'mean'])
aggregated_stats_by_workingday


Meskipun lebih banyak orang menyewa sepeda pada hari kerja, perbedaan jumlah penyewaan antara hari kerja dan hari non-kerja tidak terlalu besar. Orang masih melakukan penyewaan sepeda secara signifikan pada hari-hari non-kerja juga.

**Analisis Penyewaan Sepeda Berdasarkan Musim (Season)**

In [ ]:
# Grouping bike rental data by season
grouped_by_season = day_df.groupby('season')

# Calculating the average number of casual and registered bike rentals, as well as aggregate statistics for total bike rentals
aggregated_stats_by_season = grouped_by_season.agg({
    'casual': 'mean',
    'registered': 'mean',
    'count': ['max', 'min', 'mean']
})
aggregated_stats_by_season


Musim gugur dan musim panas menunjukkan aktivitas penyewaan sepeda yang lebih tinggi, sedangkan musim semi dan musim dingin memiliki aktivitas yang sedikit lebih rendah tetapi masih signifikan.

**Analisis Statistik Variabel Cuaca Berdasarkan Musim**

In [ ]:
# Grouping data by season and calculating aggregate statistics for temperature variables (temp),
# perceived temperature (atemp),
# and humidity (hum)
aggregated_stats_by_season = day_df.groupby('season').agg({
    'temp': ['max', 'min', 'mean'],
    'atemp': ['max', 'min', 'mean'],
    'hum': ['max', 'min', 'mean']
})
aggregated_stats_by_season


Rata-rata temperatur dan suhu rasakan tertinggi terjadi pada musim gugur (fall season), yang kemudian diikuti oleh musim panas (summer), musim dingin (winter), dan musim semi (spring).

Rata-rata kelembaban tertinggi terjadi pada musim dingin (winter season), yang kemudian diikuti oleh musim gugur (fall), musim panas (summer), dan musim semi (spring).

**Hubungan antara Penyewa Casual, Registered, dan Total**

In [ ]:
# Creating a correlation heatmap to visualize the relationship between casual, registered, and count renters
fig, ax = plt.subplots(figsize=(10, 6))
correlation_matrix = day_df.corr(numeric_only=True)
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))

sns.heatmap(
    correlation_matrix,
    annot=True,
    mask=mask,
    cmap="coolwarm",
    center=0,
    fmt=".2f"
)
plt.title("Correlation Heatmap")
plt.show()



Berdasarkan hasil analisis di atas, dapat disimpulkan beberapa pernyataan sebagai berikut:
- Variabel atemp dan temp memiliki korelasi yang sangat tinggi, yaitu sebesar 0.99, menunjukkan bahwa keduanya memiliki hubungan yang sangat erat.
- Variabel hum memiliki korelasi yang lemah dengan temp dan atemp, yaitu sebesar 0.13 dan 0.14, menunjukkan bahwa hubungan antara kelembaban udara dengan suhu dan suhu rasakan tidak begitu kuat.
- Variabel casual cukup berkorelasi dengan temp dan atemp, dengan koefisien korelasi sebesar 0.54, dan sedikit berkorelasi negatif dengan hum, yaitu sebesar -0.08.
- Variabel registered memiliki pola yang sama dengan casual, dan memiliki korelasi moderat dengan casual sebesar 0.40.
- Variabel count berkorelasi kuat dengan temp, atemp, casual, dan registered, dengan koefisien korelasi masing-masing sebesar 0.63, 0.63, 0.67, dan 0.95. Namun, variabel count memiliki korelasi sedikit negatif dengan hum, yaitu sebesar -0.10.

## Visualization & Explanatory Analysis

### Pertanyaan 1: Bagaimana tren jumlah penyewaan sepeda dalam beberapa tahun terakhir?





In [ ]:

# Grouping data by yearmonth and calculating the total casual, registered, and total rides
monthly_rent_df['total_rides'] = monthly_rent_df['casual_rides'] + monthly_rent_df['registered_rides']
fig = px.bar(monthly_rent_df,
             x='yearmonth',
             y=['casual_rides', 'registered_rides', 'total_rides'],
             barmode='group',
             color_discrete_sequence=["#FF69B4", "#00FF00", "#0000FF"],
             title="Bike Rental Trends in Recent Years",
             labels={'casual_rides': 'Casual Rentals', 'registered_rides': 'Registered Rentals', 'total_rides': 'Total Rides'})

# Displaying the figure
fig.update_layout(xaxis_title='', yaxis_title='Total Rentals',
                  xaxis=dict(showgrid=False, showline=True, linecolor='rgb(204, 204, 204)', linewidth=2, mirror=True),
                  yaxis=dict(showgrid=False, zeroline=False, showline=True, linecolor='rgb(204, 204, 204)', linewidth=2, mirror=True),
                  plot_bgcolor='rgba(255, 255, 255, 0)',
                  showlegend=True,
                  legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))

fig.show()


Berdasarkan visualisasi menggunakan bar plot, terlihat bahwa tren jumlah penyewaan sepeda dalam beberapa tahun terakhir menunjukkan peningkatan dari tahun 2011 hingga 2012. Puncak penyewaan sepeda terjadi pada bulan September di tahun 2012, sementara bulan dengan jumlah penyewaan sepeda terendah adalah bulan Januari di tahun yang sama. Di tahun 2011, puncak penyewaan sepeda terjadi pada bulan Juni, dan bulan dengan jumlah penyewaan sepeda terendah juga adalah bulan Januari. Selain itu, perbandingan antara jumlah penyewaan sepeda casual, registered, dan total juga dapat diamati dari visualisasi ini. Hal ini menunjukkan bahwa secara umum terdapat tren peningkatan jumlah penyewaan sepeda dari tahun 2011 ke tahun 2012.

### Pertanyaan 2: Apakah kondisi cuaca mempengaruhi jumlah pengguna sepeda?

In [ ]:
fig = px.box(day_df, x='weathersit', y='count', color='weathersit',
             title='Bike Users Distribution Based on Weather Condition',
             labels={'weathersit': 'Weather Condition', 'count': 'Total Rentals'})

# Display the plot
fig.show()


Dari hasil box plot dapat disimpulkan bahwa kondisi cuaca yang cerah atau sebagian cerah ("Clear/Partly Cloudy") cenderung memiliki jumlah penyewa sepeda yang lebih banyak, diikuti oleh kondisi cuaca "Misty/Cloudy", sementara kondisi cuaca "Light Snow/Rain" memiliki jumlah penyewa sepeda yang paling sedikit. Hal ini membuktikan cuaca dapat mempengaruhi atau berperan penting dalam penggunaan sepeda.

### Pertanyaan 3: Bagaimana perbedaan pola penggunaan sepeda antara hari kerja (workingday), hari libur (holiday), dan hari biasa (weekday)?





In [ ]:
# Plot for working day
fig1 = px.box(day_df, x='workingday', y='count', color='workingday',
              title='Bike Rental Clusters by Working Day',
              labels={'workingday': 'Working Day', 'count': 'Total Rentals'},
              color_discrete_sequence=['#00FFFF', '#FF00FF', '#FFFF00', '#00FF00', '#FF0000'])
fig1.update_xaxes(title_text='Working Day')
fig1.update_yaxes(title_text='Total Rentals')

# Plot for holiday
fig2 = px.box(day_df, x='holiday', y='count', color='holiday',
              title='Bike Rental Clusters by Holiday',
              labels={'holiday': 'Holiday', 'count': 'Total Rentals'},
              color_discrete_sequence=['#00FFFF', '#FF00FF', '#FFFF00', '#00FF00', '#FF0000'])
fig2.update_xaxes(title_text='Holiday')
fig2.update_yaxes(title_text='Total Rentals')

# Plot for weekday
fig3 = px.box(day_df, x='weekday', y='count', color='weekday',
              title='Bike Rental Clusters by Weekday',
              labels={'weekday': 'Weekday', 'count': 'Total Rentals'},
              color_discrete_sequence=['#00FFFF', '#FF00FF', '#FFFF00', '#00FF00', '#FF0000'])
fig3.update_xaxes(title_text='Weekday')
fig3.update_yaxes(title_text='Total Rentals')

# Displaying the plots
fig1.show()
fig2.show()
fig3.show()


Berdasarkan visualisasi diatas, diperoleh informasi sebagai berikut:
1. Jumlah penyewa sepeda lebih tinggi pada hari kerja (Senin hingga Jumat) daripada akhir pekan (Sabtu dan Minggu), di mana nilai 0 menunjukkan akhir pekan dan nilai 1 menunjukkan hari kerja.
2. Jumlah penyewa sepeda jauh lebih tinggi pada hari biasa (bukan hari libur) dibandingkan hari libur (nasional).
3. Hari Jumat memiliki jumlah penyewa sepeda tertinggi, diikuti oleh hari-hari kerja lainnya, sementara hari Minggu memiliki jumlah penyewa sepeda terendah.

### Pertanyaan 4: Apakah ada korelasi antara suhu yang menunjukkan kondisi ketika jumlah penyewaan sepeda tinggi?



In [ ]:
# Creating a scatter plot
fig = px.scatter(day_df, x='temp', y='count', color='season',
                 title='Bike Rental Clusters by Season and Temperature',
                 labels={'temp': 'Temperature (°C)', 'count': 'Total Rentals'},
                 color_discrete_sequence=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'],
                 hover_name='season')

# Displaying the plot
fig.show()



Berdasarkan scatterplot, ditemukan pola sebagai berikut:
1. Cuaca dingin pada musim dingin mengurangi jumlah penyewaan sepeda.
2. Peningkatan suhu terutama selama musim panas meningkatkan jumlah penyewaan.
3. Ada titik optimal suhu di musim gugur dan musim panas yang meningkatkan jumlah penyewaan.
4. Suhu yang nyaman pada titik optimal meningkatkan penggunaan layanan penyewaan sepeda.

### Pertanyaan 5: Apakah musim mempengaruhi jumlah penyewa sepeda berdasarkan kategori Casual dan Registered



In [ ]:
# Grouping data by season and calculating the total registered and casual usages
seasonal_usage = day_df.groupby('season')[['registered', 'casual']].sum().reset_index()

# Creating a bar plot
fig = px.bar(seasonal_usage, x='season', y=['registered', 'casual'],
             title='Bike Rental Counts by Season',
             labels={'season': 'Season', 'value': 'Total Rentals', 'variable': 'User Type'},
             color_discrete_sequence=["#00FF00","#0000FF"], barmode='group')

# Displaying the plot
fig.show()


Analisis menggunakan barplot menunjukkan bahwa aktivitas penyewaan sepeda paling tinggi terjadi selama musim gugur (Fall), diikuti oleh musim panas (Summer), musim dingin (Winter), dan terakhir musim semi (Spring). Ini berlaku baik untuk pengguna terdaftar maupun pengguna casual.

## Conclusion

- Conclution pertanyaan 1: Kondisi cuaca memang mempengaruhi jumlah pengguna sepeda. Pada hari-hari dengan kondisi cuaca yang baik, seperti cerah atau berawan, jumlah pengguna sepeda cenderung meningkat. Sebaliknya, saat kondisi cuaca buruk, seperti hujan atau salju, jumlah pengguna sepeda cenderung menurun.
- Conclution pertanyaan 2: Terdapat peningkatan secara keseluruhan dari tahun ke tahun. Pada tahun 2011, jumlah penyewaan sepeda cenderung lebih rendah dibandingkan dengan tahun 2012. Peningkatan tersebut dapat dilihat terutama pada rentang bulan Mei hingga Oktober di kedua tahun tersebut.
- Conclution pertanyaan 3: Pola penggunaan sepeda menunjukkan perbedaan yang signifikan antara hari kerja (workingday), hari libur (holiday), dan hari biasa (weekday). Jumlah pengguna sepeda cenderung lebih tinggi pada hari kerja dibandingkan dengan hari libur, meskipun perbedaannya tidak terlalu signifikan. Selain itu, hari biasa juga menunjukkan variasi dalam jumlah penggunaan sepeda, di mana Hari Jumat mencatat jumlah penggunaan tertinggi, sementara Minggu menunjukkan jumlah penggunaan terendah. Hal ini mencerminkan preferensi dan kebiasaan pengguna dalam menggunakan sepeda sesuai dengan konteks waktu dan aktivitas harian.
- Conclution pertanyaan 4:Terdapat hubungan yang jelas antara suhu dan jumlah penyewaan sepeda. Saat suhu meningkat, jumlah penyewaan sepeda juga cenderung meningkat, terutama selama musim panas. Sebaliknya, jumlah penyewaan cenderung rendah saat suhu lebih rendah, khususnya selama musim dingin.
- Conclution pertanyaan 5: Terdapat pengaruh musim terhadap jumlah penyewa sepeda baik dalam kategori Casual maupun Registered. Musim gugur dan musim panas merupakan musim dengan jumlah penyewa sepeda tertinggi, diikuti oleh musim semi dan musim dingin yang cenderung memiliki jumlah penyewa sepeda yang lebih rendah.